# Make BoW, TFIDF, Lsi

In [2]:
from tfidf_pm import *
# pubmed_corpus = PubmedCorpus()

##### load dictionary

In [ ]:
from tfidf_pm import *

##### Load LSI and TFIDF models and Dictionaries for transforming

In [ ]:
# pubmed_corpus = corpora.MmCorpus('./pmc_models_serialized/pubmed_corpus.mm')

In [ ]:
# pubmed_dict = corpora.Dictionary.load('./pmc_models_serialized/pubmed_corpus.dict')

In [ ]:
pubmed_tfidf = models.tfidfmodel.TfidfModel.load('./pmc_models_serialized/pubmed_tfidf')
# pubmed_corpus_tfidf = models.tfidfmodel.TfidfModel.load('./pmc_models_serialized/pubmed_corpus_tfidf')

In [ ]:
pubmed_lsi = models.LsiModel.load('./pmc_models_serialized/pubmed_lsi')
pubmed_corpus_lsi = models.LsiModel.load('./pmc_models_serialized/pubmed_corpus_lsi')

##### Make Similarity Matrix

In [7]:
??interfaces.SimilarityABC

In [6]:
??similarities.Similarity

In [3]:
??similarities.MatrixSimilarity

In [5]:
??models.LsiModel.load

In [ ]:
pubmed_sim = similarities.MatrixSimilarity(pubmed_corpus_lsi, pubmed_lsi.num_topics)

In [ ]:
pubmed_sim.save('./pmc_models_serialized/pubmed_sim')

### Folding in a new document and loading models

tfidf = models.tfidfmodel.TfidfModel.load('tfidf')

lsi = models.LsiModel.load('lsi')

Create an index transformer that calculates similarity based on our space
 
test_index1 = similarities.MatrixSimilarity(test_lsi1)

##### Transform New Doc

In [ ]:
new_doc = """
The abnormal position of the testis is the most common congenital anomaly of genitalia in men, with two different periods of presentation: either early in life, commonly called congenital undescended testes, or a few years later, called acquired or ascending testes. In the second group, it seems that the testis has been positioned properly at an early stage of growth, but has subsequently left its normal scrotal position to occupy the inguinal area. This group is quite controversial, as some authors considered these cases to be misdiagnoses owing to an error in physical examination.1 The cumulative experience of qualified examiners, however, suggests that this is a real phenomenon,2 which explains the increasing proportion of orchidopexies performed in late infancy.3 obesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettes"""

In [ ]:
new_doc = utils.tokenize(new_doc)

In [ ]:
new_doc_bow = pubmed_corpus_lsi.corpus.corpus.dictionary.doc2bow(new_doc)

In [ ]:
new_doc_tfidf = pubmed_tfidf[new_doc_bow]

In [ ]:
new_doc_lsi = pubmed_lsi[new_doc_tfidf]

In [ ]:
new_doc_lsi

##### Load similarity object

In [1]:
from tfidf_pm import *

In [2]:
pubmed_sim = similarities.MatrixSimilarity.load('./pmc_models_serialized/pubmed_sim')

In [3]:
pubmed_tfidf = models.tfidfmodel.TfidfModel.load('./pmc_models_serialized/pubmed_tfidf')
# pubmed_corpus_tfidf = models.tfidfmodel.TfidfModel.load('./pmc_models_serialized/pubmed_corpus_tfidf')

In [4]:
pubmed_lsi = models.LsiModel.load('./pmc_models_serialized/pubmed_lsi')
pubmed_corpus_lsi = models.LsiModel.load('./pmc_models_serialized/pubmed_corpus_lsi')

##### Load transform new doc

In [5]:
new_doc = """
The abnormal position of the testis is the most common congenital anomaly of genitalia in men, with two different periods of presentation: either early in life, commonly called congenital undescended testes, or a few years later, called acquired or ascending testes. In the second group, it seems that the testis has been positioned properly at an early stage of growth, but has subsequently left its normal scrotal position to occupy the inguinal area. This group is quite controversial, as some authors considered these cases to be misdiagnoses owing to an error in physical examination.1 The cumulative experience of qualified examiners, however, suggests that this is a real phenomenon,2 which explains the increasing proportion of orchidopexies performed in late infancy.3 obesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettesobesity obesity obesity circumference huge huge large large large cigarettes"""

In [7]:
def get_similarity_list(new_doc):
    new_doc = utils.tokenize(new_doc)
    new_doc_bow = pubmed_corpus_lsi.corpus.corpus.dictionary.doc2bow(new_doc)
    new_doc_tfidf = pubmed_tfidf[new_doc_bow]
    new_doc_lsi = pubmed_lsi[new_doc_tfidf]
    new_doc_sims = pubmed_sim[new_doc_lsi]
    return new_doc_sims

In [8]:
doc_sims = get_similarity_list(new_doc)

In [11]:
doc_sims

[(117, 0.8489375114440918),
 (78, 0.6774982213973999),
 (122, 0.64943331480026245),
 (34, 0.53155028820037842),
 (60, 0.33066171407699585),
 (4, 0.27750536799430847),
 (29, 0.16573452949523926),
 (2, 0.1274280846118927),
 (69, 0.11934912204742432),
 (70, 0.10965363681316376),
 (0, 0.10164546221494675),
 (84, 0.07911992073059082),
 (107, 0.07579571008682251),
 (45, 0.062454491853713989),
 (39, 0.062144547700881958),
 (99, 0.062088675796985626),
 (30, 0.053169988095760345),
 (101, 0.045571208000183105),
 (13, 0.041614264249801636),
 (32, 0.038229875266551971),
 (16, 0.037998057901859283),
 (93, 0.035810012370347977),
 (31, 0.035347335040569305),
 (97, 0.032695293426513672),
 (128, 0.032591566443443298),
 (64, 0.027778834104537964),
 (66, 0.021873347461223602),
 (17, 0.020937684923410416),
 (130, 0.019201306626200676),
 (38, 0.019098946824669838),
 (77, 0.018524423241615295),
 (106, 0.018408089876174927),
 (81, 0.017756335437297821),
 (119, 0.015647079795598984),
 (57, 0.01534477248787879

In [ ]:
DOCUMENT_FILE_NAMES = pubmed_corpus_lsi.corpus.corpus.document_file_names

##### Similarity one file to another

In [ ]:
import csv
import os
title_dict = {}
with open('./pmc_data/file_list.txt', 'rb') as csv_f:
    reader = csv.reader(csv_f, delimiter='\t', )
    next(reader, None)
    for row in reader:
        file_name = os.path.basename(os.path.splitext(os.path.splitext(row[0])[0])[0])
        pmc_num = row[3].replace('PMID:', '')
        title_dict[file_name] = pmc_num

In [ ]:
title_dict['Arch_Dis_Child_2007_Apr_11_92(4)_298-303']

In [ ]:
DOCUMENT_FILE_NAMES

In [ ]:
file_name = os.path.basename(os.path.splitext(DOCUMENT_FILE_NAMES[0])[0])
file_name

In [ ]:
??similarities.Similarity

In [ ]:
??interfaces.SimilarityABC

In [ ]:
??pubmed_corpus.dictionary.load

In [ ]:
??corpora.mmcorpus.MmCorpus

In [ ]:
??models.TfidfModel

In [ ]:
??corpora.TextCorpus

In [ ]:
??interfaces.CorpusABC

In [ ]:
??utils.SaveLoad

In [ ]:
dictionary = corpora.Dictionary()

In [ ]:
dictionary.doc2bow(['hello', 'something'])

In [ ]:
class thing:
    def __init__(self):
        self.array = [(1, 2, 3), (3,3, 3) , (2, 2, 2)]
    
    def __iter__(self):
        for i, j, k in self.array:
            yield i * 2
            
    def load_stuff(self):
        for i in self:
            print i
            
    def again(self):
        for i in self:
            print i

In [ ]:
a = thing()

In [ ]:
a.load_stuff()

In [ ]:
# class PubmedCorpus(corpora.TextCorpus):
    
#     def get_texts(self):
#         directory = os.walk(self.input)
#         for root, dirs, files in directory:
#             for file_name in files:
#                 file_path = os.path.join(root, file_name)
#                 with open(file_path, 'rb') as f:
#                     file_string = to_unicode_or_bust(f.read().lower())
#                     file_tokenized = word_tokenize(file_string)
#                     file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
#                     yield file_no_stops

# class PubmedCorpus(corpora.TextCorpus):
        
#     def get_texts(self):
#         for file_path in self.input:
#             with open(file_path, 'rb') as f:
#                 file_string = to_unicode_or_bust(f.read().lower())
#                 file_tokenized = word_tokenize(file_string)
#                 file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
#                 yield file_no_stops
                    
                
# def load_article_paths():
#     for root, dirs, files in os.walk('./pmc_data/pmc_text_files/'):
#         for name in files:
#             ARTICLE_FILE_PATHS.append(os.path.join(root, name))
#     genre_folders_left = len(GENRE_FOLDERS)
#     completed_genre_folders = 0 
#     for genre_folder in GENRE_FOLDERS:
#         completed_genre_folders += 1
#         genre_folder_path = os.path.join(DATA_PATH, genre_folder)
#         genre_file_list = os.listdir(genre_folder_path)
#         for article_file_title in genre_file_list:
#             article_file_path = os.path.join(genre_folder_path, article_file_title)
#             if os.path.isfile(article_file_path):
#                 ARTICLE_FILE_TITLES.append(article_file_title)
#                 ARTICLE_FILE_PATHS.append(article_file_path)
# #                 with open(article_file_path, 'rb') as f:
# #                     document = f.read()
# #                     ARTICLE_DOCUMENT_LIST.append(document)

#                 print "done with: ", article_file_title
#                 print "progress: ", completed_genre_folders / float(genre_folders_left)
#             else:
#                 sub_article_folder_list = os.listdir(article_file_path)
#                 for sub_article_file_title in sub_article_folder_list:
#                     sub_article_file_path = os.path.join(article_file_path, 
#                             sub_article_file_title)
#                     ARTICLE_FILE_TITLES.append(sub_article_file_title)
#                     ARTICLE_FILE_PATHS.append(sub_article_file_path)
# #                     with open(sub_article_file_path, 'rb') as f:
# #                         document = f.read()
# #                         ARTICLE_DOCUMENT_LIST.append(document)

### Pubmed Corpus from corpora.TextCorpus

In [ ]:
# class PubmedCorpus(corpora.TextCorpus):
#     def __init__(self, input=None, data_folder=DATA_PATH):
#         super(TextCorpus, self).__init__()
#         self.input = None
#         self.data_folder = os.walk(data_folder)
#         self.dictionary = Dictionary()
#         self.metadata = False        
#         if data_folder is not None:
#             self.dictionary.add_documents
    
#     def __iter__(self):
        
        
#     def get_texts(self, file_name):
#         with open(file_name, 'rb') as doc_file:
#             doc = doc_file.read()
#             doc_tokenized = utils.tokenize(doc, lowercase=True)
#             doc_tokenized = [word for word in doc_tokenized if word not in STOP_WORDS]
#             yield doc_tokenized

                    
# #         for doc_file in self.input:
# #             with open(doc_file, 'rb') as doc:
# #                 doc_text = doc.read()
# #                 tokenized_text = [token for token in utils.tokenize(doc_text, 
# #                                                                     lowercase=True)]
# #                 tokenized_text = [word for word in tokenized_text if word not in STOP_WORDS]
# #                 self.dictionary.add_documents([tokenized_text])
# #                 yield self.dictionary.doc2bow(tokenized_text, allow_update=False)
                
        
        
        
        
# #         directory = os.walk(self.input)
# #         for root, dirs, files in directory:
# #             for file_name in files:
# #                 file_path = os.path.join(root, file_name)
# #                 with open(file_path, 'rb') as f:
# #                     file_string = to_unicode_or_bust(f.read().lower())
# #                     file_tokenized = word_tokenize(file_string)
# #                     file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
# #                     yield file_no_stops